In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import json
import os, os.path
import importlib
import pickle

import utils.sampletools as smpl

importlib.reload(smpl)

<module 'utils.sampletools' from 'd:\\Programming\\NClassifier\\dev_tests\\tf01_cnn_detect\\utils\\sampletools.py'>

In [6]:
def generate_sample_with_target(observation, path, max_size, min_size, size_step, sample_size, sample_stride):
    def expand2tuple(x):
        if type(x) in (tuple, list, np.ndarray):
            return x
        return (x, x)

    imgregions = [smpl.transform_dictregion_to_region(x) for x in observation['regions']]

    fname = os.path.join(path, observation['imgName'])
    img = cv2.imread(fname)
    imgsize = (img.shape[1], img.shape[0])
    
    loc_max_size = expand2tuple(max_size)
    loc_min_size = expand2tuple(min_size)
    loc_size_step = expand2tuple(size_step)
    loc_sample_size = expand2tuple(sample_size)
    loc_sample_stride = expand2tuple(sample_stride)

    if imgsize[0] > loc_max_size[0] or imgsize[1] > loc_max_size[1]:
        size_factor = min(loc_max_size[0] / imgsize[0], loc_max_size[1] / imgsize[1])
        #deliberately floor in order not to exceed max_size
        imgsize = (int(np.floor(imgsize[0] * size_factor)), int(np.floor(imgsize[1] * size_factor)))

    samples = smpl.generate_sample_chain(imgsize, loc_min_size, loc_size_step, loc_sample_size, loc_sample_stride)
    targets = smpl.calculate_sample_chain_target(samples, imgregions)
    fullsample, size_chain = smpl.transform_sample_chain_to_linear_form(samples, targets)
    return fullsample, size_chain, fname

In [3]:
#src_path = '../../dev_datasets/faceimages/'
#src_file = 'eye.json'

src_path = '../../dev_datasets/pass_sample/'
src_file = 'pass_number_vertical.json'

In [4]:
with open(os.path.join(src_path, src_file), 'r') as f:
    src_sample = json.load(f)

In [12]:
%%time

desc_name = src_file.split('.')[0]
for v in src_sample.values():
    sample, size_chain, fname = generate_sample_with_target(v, src_path,
                                    max_size=1024, min_size=32, size_step=0.7071,
                                    sample_size=16, sample_stride=2)
    img_name = os.path.split(fname)[1].split('.')[0]
    sname = os.path.join(src_path,  '{0}_{1}.pickle'.format(img_name, desc_name))
    with open(sname, 'wb') as f:
        pickle.dump((sample, size_chain, fname), f)
    print('Processed {}'.format(sname))

Processed ../../dev_datasets/pass_sample/01_pass_number_vertical.pickle
Processed ../../dev_datasets/pass_sample/02_pass_number_vertical.pickle
Processed ../../dev_datasets/pass_sample/03_pass_number_vertical.pickle
Processed ../../dev_datasets/pass_sample/04_pass_number_vertical.pickle
Processed ../../dev_datasets/pass_sample/05_pass_number_vertical.pickle
Processed ../../dev_datasets/pass_sample/06_pass_number_vertical.pickle
Processed ../../dev_datasets/pass_sample/08_pass_number_vertical.pickle
Processed ../../dev_datasets/pass_sample/08b_pass_number_vertical.pickle
Processed ../../dev_datasets/pass_sample/test_vs_pass_number_vertical.pickle
Wall time: 2min 27s


In [ ]:
#pass_number_vertical.json:
#128 - 3s
#256 - 9s
#512 - 38s
#1024 - 150s